In [2]:

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import Table, func, select
from sqlalchemy.ext.hybrid import hybrid_property
from dateutil.relativedelta import relativedelta
from datetime import datetime
from flask_mail import Mail, Message
from Models.helper import u_id_generator, f_id_generator, r_id_generator
# from Models.model import User, Film, Review, Review_Like, Review_Dislike
from datetime import datetime
import bcrypt
import pandas as pd

In [3]:
app = Flask(__name__)

db_config = {
    'SQLALCHEMY_DATABASE_URI' : 'sqlite:///Database/doubi_database.db',
    'SQLALCHEMY_TRACK_MODIFICATIONS' : False
}

app.config.update(db_config)
db = SQLAlchemy(app)

In [3]:

followers = Table('followers', db.metadata,
    db.Column('followed_id', db.String(32), db.ForeignKey('user.u_id')),
    db.Column('follower_id', db.String(32), db.ForeignKey('user.u_id'))
)

blocked_users = Table('blocked_users', db.metadata,
    db.Column('blocked_id', db.String(32), db.ForeignKey('user.u_id')),
    db.Column('blocker_id', db.String(32), db.ForeignKey('user.u_id')),
)

users_wish_film = Table('users_wish_film', db.metadata,
    db.Column('user_id', db.String(32), db.ForeignKey('user.u_id')),
    db.Column('film_id', db.String(32), db.ForeignKey('film.f_id'))
)


class User(db.Model):
    __tablename__ = 'user'
    
    u_id = db.Column(db.String(32), primary_key=True, nullable=False, unique=True, default=u_id_generator)
    username = db.Column(db.String(80), nullable=False, unique=True)
    password_hash = db.Column(db.Text, nullable=False)
    email = db.Column(db.String(80), nullable=False, unique=True)
    url_photo = db.Column(db.Text, nullable=True)
    is_admin = db.Column(db.Boolean, nullable=False, default=False)
    is_blocked = db.Column(db.Boolean, nullable=False, default=False)

    created_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)
    updated_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow, onupdate=datetime.utcnow)

    followed = db.relationship('User', 
                                secondary=followers,
                                primaryjoin=(followers.c.follower_id == u_id),
                                secondaryjoin=(followers.c.followed_id == u_id),
                                backref=db.backref('followers', lazy='dynamic'),
                                lazy='dynamic')

    blocked = db.relationship('User',
                                secondary=blocked_users,
                                primaryjoin=(blocked_users.c.blocker_id == u_id),
                                secondaryjoin=(blocked_users.c.blocked_id == u_id),
                                backref=db.backref('blockers', lazy='dynamic'),
                                lazy='dynamic')
    
    reviews = db.relationship('Review', backref='user', lazy='dynamic')

    review_likes = db.relationship('Review_Like', backref='user', lazy='dynamic')
    review_dislikes = db.relationship('Review_Dislike', backref='user', lazy='dynamic')
    
    wish = db.relationship('Film', secondary=users_wish_film, backref='user', lazy='dynamic')
    
    @property
    def password(self):
        raise AttributeError('password is not a readable attribute')

    @password.setter
    def password(self, password):
        salt = bcrypt.gensalt()
        self.password_hash = bcrypt.hashpw(password.encode('utf-8'), salt)
    
    def verify_password(self, password):
        return bcrypt.checkpw(password.encode('utf-8'), self.password_hash)
    
    def __repr__(self):
        return '<User %r>' % self.username


    

class Film(db.Model):
    __tablename__ = 'film'

    f_id = db.Column(db.String(32), primary_key=True, nullable=False, unique=True, default=f_id_generator)
    title = db.Column(db.String(80), nullable=False)
    genre = db.Column(db.String(80), nullable=False)
    year = db.Column(db.Integer, nullable=True)
    run_time = db.Column(db.String(16), nullable=True)
    rating_imdb = db.Column(db.Float, nullable=True)
    overview = db.Column(db.String(500), nullable=True)
    director = db.Column(db.String(80), nullable=True)
    actor = db.Column(db.String(200), nullable=True)
    url_poster = db.Column(db.Text, nullable=True)
    rating_doubi = db.Column(db.Float, nullable=True)

    created_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)
    updated_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow, onupdate=datetime.utcnow)

    reviews = db.relationship('Review', backref='film', lazy='dynamic')
    
    @hybrid_property
    def rating(self):
        reviews = self.reviews.all()
        if len(reviews) == 0:
            return 0
        else:
            return round(sum(review.rating for review in reviews) / len(reviews), 1)
        
    @rating.expression
    def rating(cls):
        return select(func.avg(Review.rating)).where(Review.f_id == cls.f_id)
    
    @property
    def rating_distribution(self):
        reviews = self.reviews.all()
        rating_distribution = {x: 0 for x in range(0, 5)}
        for review in reviews:
            rating_distribution[review.rating] = rating_distribution.get(review.rating, 0) + 1
        return rating_distribution

    @property
    def genres(self):
        return [genre.strip() for genre in self.genre.split(',')]
    
    @property
    def actors(self):
        return [actor.strip() for actor in self.actor.split(',')]
    
    def __repr__(self):
        return '<Film %r>' % self.title
    


class Review(db.Model):
    __tablename__ = 'review'

    r_id = db.Column(db.String(32), primary_key=True, nullable=False, unique=True, default=r_id_generator)
    u_id = db.Column(db.String(32), db.ForeignKey('user.u_id'), nullable=False)
    f_id = db.Column(db.String(32), db.ForeignKey('film.f_id'), nullable=False)
    content = db.Column(db.String(500), nullable=True)
    rating = db.Column(db.Integer, nullable=False)
    bad_word = db.Column(db.Boolean, nullable=False, default=False)

    created_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)
    updated_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow, onupdate=datetime.utcnow)

    likes = db.relationship('Review_Like', backref='review', lazy='dynamic')
    dislikes = db.relationship('Review_Dislike', backref='review', lazy='dynamic')


class Review_Like(db.Model):
    __tablename__ = 'review_like'

    r_id = db.Column(db.String(32), db.ForeignKey('review.r_id'), primary_key=True, nullable=False)
    u_id = db.Column(db.String(32), db.ForeignKey('user.u_id'), primary_key=True, nullable=False)

    created_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)


class Review_Dislike(db.Model):
    __tablename__ = 'review_dislike'

    r_id = db.Column(db.String(32), db.ForeignKey('review.r_id'), primary_key=True, nullable=False)
    u_id = db.Column(db.String(32), db.ForeignKey('user.u_id'), primary_key=True, nullable=False)

    created_time = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)

In [4]:
# from sqlite3 import connect
# conn = connect(app.root_path + '/Database/doubi_database_old.db')
# c = conn.cursor()

# c.execute('SELECT * FROM film')
# films = c.fetchall()
# print(films)
db.create_all()

In [7]:
from sqlite3 import connect
from sqlalchemy import insert
def data_transfer():
    
    table_dict = {
        "followers": followers,
        "users_wish_film": users_wish_film,
        "blocked_users" : blocked_users,
    }
    
    conn = connect(app.root_path + '/Database/doubi_database_old.db')
    c = conn.cursor()
    
    # get all table name
    c.execute('SELECT name FROM sqlite_master WHERE type="table"')
    tables = c.fetchall()
    for table in tables:
        table = table[0]
        print("----- processing table: <%s> -----" % table)
        
        c.execute('SELECT * FROM %s' % table)
        data_old = c.fetchall()
        
        c.execute('PRAGMA table_info(%s)' % table)
        attributes = c.fetchall()
        
        attributes = [attribute[1] for attribute in attributes]
        data = []
        if table == 'user':
            continue
            attributes[2] = 'password'
            
            for index, item in enumerate(data_old):
                print("%s(%d/%d)" % (item[1], index+1, len(data_old)))
                item = list(item)
                item[2] = item[1][0] + '_DOUBI123'
                item[7] = datetime.strptime(item[7], '%Y-%m-%d %H:%M:%S.%f')
                item[8] = datetime.strptime(item[8], '%Y-%m-%d %H:%M:%S.%f')
                
                db.session.add(User(
                    u_id=item[0],
                    username=item[1],
                    password=item[2],
                    email=item[3],
                    url_photo=item[4],
                    is_admin=item[5],
                    is_blocked=item[6],
                    created_time=item[7],
                    updated_time=item[8]
                ))
            db.session.commit()
            print("----- <user> done -----")
        elif table == 'film':
            continue
            data = []
            for index, item in enumerate(data_old):
                print("%s(%d/%d)" % (item[1], index+1, len(data_old)))
                item = list(item)
                item[attributes.index('created_time')] = datetime.strptime(item[attributes.index('created_time')], '%Y-%m-%d %H:%M:%S.%f')
                item[attributes.index('updated_time')] = datetime.strptime(item[attributes.index('updated_time')], '%Y-%m-%d %H:%M:%S.%f')
                
                data.append(dict(zip(attributes, item)))
                
            db.session.bulk_insert_mappings(Film, data)
            db.session.commit()
            print("----- <%s> done -----" % table)
        elif table == 'review':
            continue
            data = []
            for inde, item in enumerate(data_old):
                print("%s(%d/%d)" % (table, inde+1, len(data_old)))
                item = list(item)
                item[attributes.index('created_time')] = datetime.strptime(item[attributes.index('created_time')], '%Y-%m-%d %H:%M:%S.%f')
                item[attributes.index('updated_time')] = datetime.utcnow()
                
                data.append(dict(zip(attributes, item)))
            
            db.session.bulk_insert_mappings(Review, data)
            db.session.commit()
            print("----- <%s> done -----" % table)
        elif table in ["review_like", "review_dislike"]:
            continue
            data = []
            for index, item in enumerate(data_old):
                print("%s(%d/%d)" % (table, index+1, len(data_old)))
                item = list(item)
                item[attributes.index('created_time')] = datetime.strptime(item[attributes.index('created_time')], '%Y-%m-%d %H:%M:%S.%f')
                
                data.append(dict(zip(attributes, item)))
            
            if table == "review_Like":
                db.session.bulk_insert_mappings(Review_Like, data)
            else:
                db.session.bulk_insert_mappings(Review_Dislike, data)
            db.session.commit()
            print("----- <%s> done -----" % table)
        else:
            data = []
            # insert data to table named <table>
            for index, item in enumerate(data_old):
                print("%s(%d/%d)" % (table, index+1, len(data_old)))
                item = list(item)
                data.append(dict(zip(attributes, item)))
                
                db.session.execute(insert(table_dict[table]).values(item))
                
            db.session.commit()
            print("----- <%s> done -----" % table)

In [ ]:
db.session.rollback()
data_transfer()

In [4]:
# create a table named bad_word
bad_word = db.Table('bad_word', db.metadata,
    db.Column('w_id', db.Integer, primary_key=True, autoincrement=True),
    db.Column('word', db.String(32), nullable=False))

db.create_all()
                    

In [11]:
words = pd.read_csv("D:/Users/zzy80/Desktop/bad-words.csv", header=None)

# insert the data to table bad_word
for index, word in words.iterrows():
    db.session.execute(bad_word.insert().values(word=word[0]))
db.session.commit()

In [8]:
db.session.query(bad_word.c.word).all()

[('jigaboo',),
 ('mound of venus',),
 ('asslover',),
 ('s&m',),
 ('queaf',),
 ('whitetrash',),
 ('meatrack',),
 ('ra8s',),
 ('pimp',),
 ('urine',),
 ('whit',),
 ('randy',),
 ('herpes',),
 ('niglet',),
 ('narcotic',),
 ('pudboy',),
 ('rimming',),
 ('boner',),
 ('pornography',),
 ('poop chute',),
 ('israel',),
 ('dong',),
 ('slanteye',),
 ('muffdiving',),
 ('jiggabo',),
 ('assassination',),
 ('peepshpw',),
 ('popimp',),
 ('girl on',),
 ('testicles',),
 ('laid',),
 ('molestor',),
 ('peni5',),
 ('tranny',),
 ('barface',),
 ('hell',),
 ('arsehole',),
 ('pissed',),
 ('sixsixsix',),
 ('execute',),
 ('shitty ',),
 ('conspiracy',),
 ('hamas',),
 ('cunilingus',),
 ('bitcher',),
 ('muslim',),
 ('pee',),
 ('niggled',),
 ('muffindiver',),
 ('cocksman',),
 ('scag',),
 ('aroused',),
 ('niggling',),
 ('fingerfucker ',),
 ('nlggor',),
 ('niggaz',),
 ('assfuck',),
 ('slant',),
 ('urinate',),
 ('mothafucked ',),
 ('fungus',),
 ('retard',),
 ('gummer',),
 ('venus mound',),
 ('alla',),
 ('spaghettinigger',